In [1]:
##计算销售价格##
import pandas as pd
import numpy as np
df1 = pd.read_excel('附件2.xlsx',sheet_name='2023年统计的相关数据')
df1[['min_price', 'max_price']] = df1['销售单价/(元/斤)'].str.split('-', expand=True)
# 将拆分出的字符串转换为整数
df1['min_price'] = df1['min_price'].astype(float)
df1['max_price'] = df1['max_price'].astype(float)
def estimate_lognormal_params(min_price, max_price):
    mu = np.log(min_price)
    sigma = np.log(max_price / min_price) / 2
    return mu, sigma
# 生成对数正态分布的样本价格
def generate_lognormal_price(mu, sigma):
    return np.exp(np.random.normal(mu, sigma))

# 应用到 DataFrame
np.random.seed(0)  # 设定随机种子以确保结果可重复

df1['mu'], df1['sigma'] = zip(*df1.apply(lambda row: estimate_lognormal_params(row['min_price'], row['max_price']), axis=1))
df1['estimated_price'] = df1.apply(lambda row: generate_lognormal_price(row['mu'], row['sigma']), axis=1)

# 删除中间计算的列和冗余列
df1.drop(columns=['mu', 'sigma'], inplace=True)
df2=df1.drop(['序号','min_price','max_price','销售单价/(元/斤)'],axis=1)
##美化数据集
df2 = df2.rename(columns={ 'estimated_price': '销售单价'})
df2.head()

,作物编号,作物名称,地块类型,种植季次,亩产量/斤,种植成本/(元/亩),销售单价
0,1,黄豆,平旱地,单季,400.0,400.0,3.784244
1,2,黑豆,平旱地,单季,500.0,400.0,6.858413
2,3,红豆,平旱地,单季,400.0,350.0,8.199929
3,4,绿豆,平旱地,单季,350.0,350.0,8.282061
4,5,爬豆,平旱地,单季,415.0,350.0,7.389989


In [2]:
##预测未来几年的亩产量，销售单价以及种植成本。
# 初始数据
years = np.arange(2024, 2031)
n_simulations = 1000

In [3]:
df3=pd.read_excel('附件2.xlsx',sheet_name='2023年的农作物种植情况')
df3 = df3.drop_duplicates(subset='作物名称', keep='first')
df3=df3[['作物名称','作物类型']]
df4=pd.merge(df2, df3[['作物名称', '作物类型']],
                  on=['作物名称'],
                  how='left')
#预测销售价格
def judge_price(x,y):
    if x=='粮食' or x=='粮食（豆类） ':
        g = 0
    elif x=='蔬菜 ' or x=='蔬菜（豆类）':
        g= np.random.uniform(0.05, 0.10)
    elif x=='食用菌' and y!='羊肚菌':
        g= -np.random.uniform(0.01, 0.05)
    else :
        g=-0.05
    return g
years = np.arange(2024, 2031)
n_simulations = 1000
#1000次随机
for year in years:
    for i in range(len(df4)):
        result=[]
        price=df4.at[i,'销售单价']
        for _ in range(n_simulations):
            x=df4.at[i,'作物类型']
            y=df4.at[i,'作物名称']
            g=judge_price(x,y)
            result.append(price*(1+g))
        average = sum(result) / len(result)
        name=str(year)+'_'+'销售单价'
        df4.at[i,name]=average
df4 = df4.drop(df4.tail(3).index)

In [4]:
#预测亩产量
for year in years:
    for i in range(len(df4)):
        result=[]
        yield_per_mu=df4.at[i,'亩产量/斤']
        for _ in range(n_simulations):
            g= np.random.uniform(0.90, 1.10)
            result.append(yield_per_mu*(1+g))
        average = sum(result) / len(result)
        name=str(year)+'_'+'亩产量/斤'
        df4.at[i,name]=average

In [5]:
#预测销售成本
for year in years:
    for i in range(len(df4)):
        result=[]
        cost=df4.at[i,'种植成本/(元/亩)']
        for _ in range(n_simulations):
            g= np.random.uniform(0, 0.05)
            result.append(yield_per_mu*(1+g))
        average = sum(result) / len(result)
        name=str(year)+'_'+'种植成本/(元/亩)'
        df4.at[i,name]=average

In [6]:
#预测销售量
data1 = pd.read_excel('附件2.xlsx',sheet_name='2023年统计的相关数据')
data2 = pd.read_excel('附件2.xlsx',sheet_name='2023年的农作物种植情况')
result = pd.merge(data2, data1[['作物名称', '地块类型', '亩产量/斤', '种植成本/(元/亩)']],
                  on=['作物名称'],
                  how='left')
result['预测销售量']=result['亩产量/斤']*result['种植面积/亩']
result=result.groupby('作物名称')['预测销售量'].sum()
result_df = pd.DataFrame(result.reset_index())  
result_df.columns = ['作物名称', '预期销售量']
result_df = result_df[ result_df['作物名称']!='生菜']
result_df['预期销售量'] = result_df['预期销售量'].astype(int)  
result_df.reset_index(drop=True, inplace=True)
def judge_s(x):
    if x=='小麦' or x=='玉米':
        g = np.random.uniform(0.05, 0.10)
    else :
        g= np.random.uniform(-0.05, 0.05)
    return g
print(result_df['预期销售量'])
for year in years:
    for i in range(len(result_df)):
        result1=[]
        s=result_df.at[i,'预期销售量']
        for _ in range(n_simulations):
            x=result_df.at[i,'作物名称']
            g= judge_s(x)
            result1.append(s*(1+g))
        average = int(sum(result1) / len(result1))
        name=str(year)+'_'+'预期销售量'
        result_df.at[i,name]=average

0      87120
1      11070
2     111150
3      99000
4     150000
5      30000
6     117720
7     506160
8       9000
9      21000
10     12240
11     29625
12    384750
13      4080
14     18000
15    100000
16      9900
17     36000
18    113400
19     68400
20     95520
21      4200
22     17640
23      5430
24    149040
25      4725
26     42000
27      9810
28      2700
29    120690
30    210900
31    115640
32      3240
33      7290
34      7200
35     90000
36      4920
37     36450
38    167580
39     37500
40     65550
Name: 预期销售量, dtype: int64


In [7]:
df5 = pd.merge(df4,result_df,on=['作物名称'])

In [8]:
file_2 = '附件2.xlsx'
file_2_data = pd.ExcelFile(file_2)
# 读取2023年农作物种植情况和2023年统计的相关数据
crop_data_2023 = file_2_data.parse('2023年的农作物种植情况')
crop_statistics_2023 = file_2_data.parse('2023年统计的相关数据')
# 提取关键字段用于分析
crop_summary = crop_statistics_2023[['作物编号', '作物名称', '亩产量/斤', '种植成本/(元/亩)']]
# 将2023年种植数据与作物统计数据进行合并
merged_data = pd.merge(crop_data_2023, crop_summary, on='作物编号', how='left')

In [9]:
import pulp

# 初始化问题（最大化收益）
model = pulp.LpProblem("Maximize_Crop_Profit_Discount", pulp.LpMaximize)

# 获取实际数据
plot_codes = merged_data['种植地块'].unique()  # 地块编码
crop_names = merged_data['作物名称_x'].unique()  # 作物名称

# 从 merged_data 生成 land_area_dict，确保每个地块都有正确的面积
land_area_dict = merged_data.set_index('种植地块')['种植面积/亩'].to_dict()

# 使用前向填充方法来填充空缺值，确保数据完整性
merged_data.fillna(method='ffill', inplace=True)

# 清理地块编码，确保只有字符串类型的地块编码被处理
plot_codes_clean = [str(plot).strip() if isinstance(plot, str) else str(plot) for plot in plot_codes]

# 筛选出豆类作物
bean_crops = merged_data[merged_data['作物类型'].isin(['粮食（豆类）', '蔬菜（豆类）'])]['作物名称_x'].unique()

# 初始化每个地块的计数器，记录每块地连续多少年没有种植豆类
bean_counter = {plot: 0 for plot in plot_codes_clean}

# 初始化字典来记录上一年每块地种植的作物
previous_year_crops = {}

# 筛选出地块类型
abc_plots = [plot for plot in plot_codes_clean if plot[0] in ['A', 'B', 'C']]
water_irrigated_plots = [plot for plot in plot_codes_clean if plot[0] == 'D']
restricted_crops = ['大白菜', '白萝卜', '红萝卜']
normal_greenhouse_plots = [plot for plot in plot_codes_clean if plot[0] == 'E']
# 筛选出智慧大棚的地块（假设智慧大棚的标识为F）
smart_greenhouse_plots = [plot for plot in plot_codes_clean if plot[0] == 'F']
# 筛选出作物类型为食用菌的作物
mushroom_crops = merged_data[merged_data['作物类型'] == '食用菌']['作物名称_x'].unique()
# 筛选出作物类型为蔬菜的作物
vegetable_data = merged_data[merged_data['作物类型'] == '蔬菜']

# 定义两个季度
seasons = [1, 2]
results = []

crop_binary_vars = pulp.LpVariable.dicts("CropBinaryVars",((plot, crop) for plot in plot_codes_clean for crop in crop_names),cat='Binary')
grain_crops = [crop for crop in crop_names if '水稻' not in crop and '粮食' in merged_data[merged_data['作物名称_x'] == crop]['作物类型'].values]

C:\Users\18798\AppData\Local\Temp\ipykernel_28612\3158479218.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='ffill', inplace=True)
C:\Users\18798\AppData\Local\Temp\ipykernel_28612\3158479218.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data.fillna(method='ffill', inplace=True)


In [10]:
# 循环优化每年的种植方案（2023-2030）
for year in range(2024, 2031):
    print(f"Optimizing for the year: {year}")

    # 提取当年的亩产量、种植成本、销售单价、预期销售量
    crop_yield = df5[f'{year}_亩产量/斤'].tolist()  # 每种作物的亩产量
    crop_cost = df5[f'{year}_种植成本/(元/亩)'].tolist()  # 每种作物的种植成本
    crop_price = df5[f'{year}_销售单价'].tolist()  # 每种作物的销售价格
    expected_sales = df5[f'{year}_预期销售量'].tolist()  # 每种作物的预期销售量


    # 定义正常销售量 y_normal_{plot_code, crop, season} 和折扣销售量 y_discounted_{plot_code, crop, season}
    sales_normal = pulp.LpVariable.dicts(f"y_normal_{year}",
                                        ((plot, crop, season) for plot in plot_codes_clean for crop in crop_names for season in seasons),
                                        lowBound=0, cat='Continuous')

    sales_discounted = pulp.LpVariable.dicts(f"y_discounted_{year}",
                                            ((plot, crop, season) for plot in plot_codes_clean for crop in crop_names for season in seasons),
                                            lowBound=0, cat='Continuous')

    # 定义每块地每季度的种植决策变量 x_{plot_code, crop, season}: 每块地 i 在季节 season 种植作物 j 的面积
    crop_vars = pulp.LpVariable.dicts(f"x_{year}",
                                      ((plot, crop, season) for plot in plot_codes_clean for crop in crop_names for season in seasons),
                                      lowBound=0, cat='Continuous')

    # 定义实际销售量 y_{plot_code, crop, season} 用于处理滞销问题
    sales_vars = pulp.LpVariable.dicts(f"y_{year}",
                                       ((plot, crop, season) for plot in plot_codes_clean for crop in crop_names for season in seasons),
                                       lowBound=0, cat='Continuous')

     # 目标函数：最大化每季的收益,增加折价销售部分
    model += pulp.lpSum([sales_normal[plot, crop, season] * crop_price[j] 
                     + sales_discounted[plot, crop, season] * (0.5 * crop_price[j])
                     - crop_vars[plot, crop, season] * crop_cost[j]
                     for plot in plot_codes_clean 
                     for crop, j in zip(crop_names, range(len(crop_names))) 
                     for season in seasons])

    # 添加销售量约束
    for plot in plot_codes_clean:
        for crop, j in zip(crop_names, range(len(crop_names))):
            for season in seasons:
                # 正常销售部分不能超过预期销售量，约束名称加入年份，确保唯一性
                model += sales_normal[plot, crop, season] <= expected_sales[j], f"Sales_Normal_Limit_{plot}_{crop}_{season}_{year}"

                # 总销售量（正常销售+折价销售）不能超过实际产量
                model += sales_normal[plot, crop, season] + sales_discounted[plot, crop, season] <= crop_vars[plot, crop, season] * crop_yield[j], f"Total_Sales_Limit_{plot}_{crop}_{season}_{year}"

                # 折价销售部分为超过预期销售量的部分，使用两条约束来模拟 max(0, 产量 - 预期销售量)
                model += sales_discounted[plot, crop, season] >= 0, f"Discounted_Sales_NonNegative_{plot}_{crop}_{season}_{year}"
                model += sales_discounted[plot, crop, season] >= (crop_vars[plot, crop, season] * crop_yield[j] - expected_sales[j]), f"Discounted_Sales_Limit_{plot}_{crop}_{season}_{year}"


    # 为每个地块添加种植面积总量的约束
    for plot in plot_codes_clean:
        # 计算地块的总种植面积：包括第一季和第二季的所有作物
        total_area_planted = pulp.lpSum([crop_vars[plot, crop, season] for crop in crop_names for season in seasons])
        # 确保总种植面积不能超过该地块的总面积
        model += total_area_planted <= land_area_dict.get(plot, 0), f"Total_Area_Limit_{plot}_{year}"

    # 添加约束：每块地最多种植 3 种作物
    max_crops_per_plot = 1
    for plot in plot_codes_clean:
        # 限制每块地最多种植 max_crops_per_plot 种作物
        model += pulp.lpSum([crop_binary_vars[plot, crop] for crop in crop_names]) <= max_crops_per_plot, f"MaxCrops_{plot}_{year}"

    # 添加约束：实际销售量不能超过预期销售量和种植产量
    for plot in plot_codes_clean:
        for crop, j in zip(crop_names, range(len(crop_names))):
            for season in seasons:
                model += sales_vars[plot, crop, season] <= expected_sales[j], f"Sales_Limit_{plot}_{crop}_{season}_{year}"
                model += sales_vars[plot, crop, season] <= crop_vars[plot, crop, season] * crop_yield[j], f"Yield_Limit_{plot}_{crop}_{season}_{year}"

    # 每块地每季度的种植面积不能超过其总面积
    for plot in plot_codes_clean:
        for season in seasons:
            model += pulp.lpSum([crop_vars[plot, crop, season] for crop in crop_names]) <= land_area_dict.get(plot, 0), f"Land_Limit_{plot}_{season}_{year}"
    # 为平旱地、梯田和山坡地的地块添加约束
    for plot in abc_plots:
        for crop in crop_names:
            # 限制只能种植粮食类作物（不包括水稻）
            if crop in grain_crops:
                # 只允许第一季种植作物，第二季种植面积必须为 0
                model += crop_vars[plot, crop, 2] == 0, f"ABC_Plot_Season2_Zero_{plot}_{crop}_{year}"
            else:
                # 不能种植非粮食作物
                model += crop_vars[plot, crop, 1] == 0, f"ABC_Plot_Grain_Only_{plot}_{crop}_{year}"

    # 在ABC类型地块的season2中不种植作物
    for plot in abc_plots:
        for crop in crop_names:
            model += crop_vars[plot, crop, 2] == 0, f"Season2_Zero_Limit_{plot}_{crop}_{year}"

    # 添加连续两年不能种植同一种作物的约束，按季度进行
    if year > 2023:
        for plot in plot_codes_clean:
            if plot in previous_year_crops:
                for crop in crop_names:
                    for season in seasons:
                        # 如果上一年相同季度种植了某种作物，今年同一季度不能种同一种作物
                        model += crop_vars[plot, crop, season] <= (1 - previous_year_crops[plot][crop, season]) * land_area_dict.get(plot, 0), f"Rotation_Limit_{plot}_{crop}_{season}_{year}"

    # 对每块地的豆类作物进行检查，按季度处理
    for plot in plot_codes_clean:
        # 定义一个二元变量，表示是否种植了豆类作物
        bean_planted_var = pulp.LpVariable(f"bean_planted_{plot}_{year}", 0, 1, cat='Binary')

        # 如果种植的豆类作物面积大于 0，bean_planted_var 必须为 1
        model += pulp.lpSum([crop_vars[plot, crop, season] for crop in bean_crops for season in seasons]) >= 1 * bean_planted_var, f"Bean_Planted_Constraint_{plot}_{year}"

        # 如果种植了豆类，重置计数器
        if bean_planted_var == 1:
            bean_counter[plot] = 0
        else:
            # 如果没有种植豆类，计数器 +1
            bean_counter[plot] += 1

        # 如果计数器达到 3，则强制种植豆类作物，并重置计数器
        if bean_counter[plot] >= 3:
            print(f"Plot {plot} has not planted beans for 3 years, forcing bean crop.")
            model += pulp.lpSum([crop_vars[plot, crop, season] for crop in bean_crops for season in seasons]) >= 10, f"Force_Bean_Crop_{plot}_{year}"
            bean_counter[plot] = 0

        # 添加新的约束：水浇地每年可以单季种植水稻或两季种植蔬菜作物
    for plot in water_irrigated_plots:
        for crop in crop_names:
            for season in seasons:
                if '水稻' in crop:
                    # 如果种植水稻，season2 全部设置为 0
                    model += crop_vars[plot, crop, 2] == 0, f"Water_Irrigated_Season2_Zero_{plot}_{crop}_Season_{season}_Year_{year}"
                elif '蔬菜' in crop:
                    # 第一季可以种植任意蔬菜，除大白菜、白萝卜和红萝卜外
                    if season == 1 and crop not in restricted_crops:
                        model += crop_vars[plot, crop, season] >= 0, f"Water_Irrigated_Season1_Veg_{plot}_{crop}_Season_{season}_Year_{year}"
                    # 第二季只能种植大白菜、白萝卜和红萝卜
                    if season == 2 and crop in restricted_crops:
                        model += crop_vars[plot, crop, season] >= 0, f"Water_Irrigated_Season2_RestrictedVeg_{plot}_{crop}_Season_{season}_Year_{year}"

    # 确保第二季只能种植大白菜、白萝卜和红萝卜中的一种
    for plot in water_irrigated_plots:
        # 确保只能种植一种受限制的作物
        model += pulp.lpSum([crop_vars[plot, crop, 2] for crop in restricted_crops]) <= land_area_dict.get(plot, 0), f"Water_Irrigated_Season2_OneCrop_{plot}_Year_{year}"

    # 添加普通大棚的种植约束
    for plot in normal_greenhouse_plots:
        for crop in crop_names:
            for season in seasons:
                if season == 1 and crop not in restricted_crops:
                    # 第一季可以种植任何蔬菜，除大白菜、白萝卜和红萝卜外
                    model += crop_vars[plot, crop, season] >= 0, f"NormalGreenhouse_Season1_Veg_{plot}_{crop}_Season_{season}_Year_{year}"
                elif season == 2 and crop in mushroom_crops:
                    # 第二季只能种植食用菌
                    model += crop_vars[plot, crop, season] >= 0, f"NormalGreenhouse_Season2_Mushroom_{plot}_{crop}_Season_{season}_Year_{year}"
    # 确保第二季只能种植食用菌
    for plot in normal_greenhouse_plots:
        # 确保只能种植食用菌
        model += pulp.lpSum([crop_vars[plot, crop, 2] for crop in mushroom_crops]) <= land_area_dict.get(plot, 0), f"NormalGreenhouse_Season2_OnlyMushroom_{plot}_Year_{year}"
        # 筛选出不包含大白菜、白萝卜和红萝卜的蔬菜作物
    allowed_vegetable_crops = vegetable_data[~vegetable_data['作物名称_x'].isin(restricted_crops)]['作物名称_x'].unique()



    # 应用智慧大棚的种植约束：每年可以种植两季蔬菜，除大白菜、白萝卜和红萝卜外
    for plot in smart_greenhouse_plots:
        for crop in allowed_vegetable_crops:
            for season in seasons:
                # 第一季和第二季都可以种植任意蔬菜，除大白菜、白萝卜和红萝卜外
                model += crop_vars[plot, crop, season] >= 0, f"SmartGreenhouse_Season{season}_Veg_{plot}_{crop}_Year_{year}"


    # 求解每一年的优化问题
    model.solve()

    # 在求解之后保存当前年份的种植作物信息，用于下一个年份的轮作限制
    for plot in plot_codes_clean:
        previous_year_crops[plot] = {(crop, season): (crop_vars[plot, crop, season].varValue > 0) for crop in crop_names for season in seasons}

    # 输出每年优化结果
    # print(f"Year {year} results:")
    for plot in plot_codes_clean:
        for crop in crop_names:
            for season in seasons:
                print(f"Plot {plot}, Crop {crop}, Season {season}, Area: {crop_vars[plot, crop, season].varValue}")
    for plot in plot_codes_clean:  # 遍历地块
        for crop in crop_names:  # 遍历作物
            for season in seasons:  # 遍历季次（1, 2）
                # 获取每块地每季的种植面积
                area = crop_vars[plot, crop, season].varValue
                # 如果有种植面积大于0的作物，存储结果
                if area is not None and area > 0:
                    results.append([year, plot, season, crop, area])
# 将结果列表转换为 DataFrame
df_results = pd.DataFrame(results, columns=['年份', '地块', '季次', '作物名称', '种植面积'])

# 查看优化结果 DataFrame
print(df_results)
# 输出总收益
print(f"Maximized Profit for {2023}-{2030}: {pulp.value(model.objective)}")

Optimizing for the year: 2024
Plot A1, Crop 小麦, Season 1, Area: 0.0
Plot A1, Crop 小麦, Season 2, Area: 0.0
Plot A1, Crop 玉米, Season 1, Area: 0.0
Plot A1, Crop 玉米, Season 2, Area: 0.0
Plot A1, Crop 黄豆, Season 1, Area: 0.0
Plot A1, Crop 黄豆, Season 2, Area: 0.0
Plot A1, Crop 绿豆, Season 1, Area: 0.0
Plot A1, Crop 绿豆, Season 2, Area: 0.0
Plot A1, Crop 谷子, Season 1, Area: 0.0
Plot A1, Crop 谷子, Season 2, Area: 0.0
Plot A1, Crop 黑豆, Season 1, Area: 0.0
Plot A1, Crop 黑豆, Season 2, Area: 0.0
Plot A1, Crop 红豆, Season 1, Area: 0.0
Plot A1, Crop 红豆, Season 2, Area: 0.0
Plot A1, Crop 爬豆, Season 1, Area: 0.0
Plot A1, Crop 爬豆, Season 2, Area: 0.0
Plot A1, Crop 高粱, Season 1, Area: 0.0
Plot A1, Crop 高粱, Season 2, Area: 0.0
Plot A1, Crop 黍子, Season 1, Area: 35.646547
Plot A1, Crop 黍子, Season 2, Area: 0.0
Plot A1, Crop 莜麦, Season 1, Area: 0.0
Plot A1, Crop 莜麦, Season 2, Area: 0.0
Plot A1, Crop 大麦, Season 1, Area: 18.546671
Plot A1, Crop 大麦, Season 2, Area: 0.0
Plot A1, Crop 荞麦, Season 1, Area: 25.806782
Pl

c:\Users\18798\AppData\Local\Programs\Python\Python311\Lib\site-packages\pulp\pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Plot A1, Crop 小麦, Season 1, Area: 0.0
Plot A1, Crop 小麦, Season 2, Area: 0.0
Plot A1, Crop 玉米, Season 1, Area: 0.0
Plot A1, Crop 玉米, Season 2, Area: 0.0
Plot A1, Crop 黄豆, Season 1, Area: 0.0
Plot A1, Crop 黄豆, Season 2, Area: 0.0
Plot A1, Crop 绿豆, Season 1, Area: 0.0
Plot A1, Crop 绿豆, Season 2, Area: 0.0
Plot A1, Crop 谷子, Season 1, Area: 0.0
Plot A1, Crop 谷子, Season 2, Area: 0.0
Plot A1, Crop 黑豆, Season 1, Area: 0.0
Plot A1, Crop 黑豆, Season 2, Area: 0.0
Plot A1, Crop 红豆, Season 1, Area: 0.0
Plot A1, Crop 红豆, Season 2, Area: 0.0
Plot A1, Crop 爬豆, Season 1, Area: 0.0
Plot A1, Crop 爬豆, Season 2, Area: 0.0
Plot A1, Crop 高粱, Season 1, Area: 71.450868
Plot A1, Crop 高粱, Season 2, Area: 0.0
Plot A1, Crop 黍子, Season 1, Area: 0.0
Plot A1, Crop 黍子, Season 2, Area: 0.0
Plot A1, Crop 莜麦, Season 1, Area: 8.549132
Plot A1, Crop 莜麦, Season 2, Area: 0.0
Plot A1, Crop 大麦, Season 1, Area: 0.0
Plot A1, Crop 大麦, Season 2, Area: 0.0
Plot A1, Crop 荞麦, Season 1, Area: 0.0
Plot A1, Crop 荞麦, Season 2, Area: 0.0
P

In [11]:
import pandas as pd
from openpyxl import load_workbook

df_results = pd.DataFrame(results, columns=['年份', '地块', '季次', '作物名称', '种植面积'])
def filter_rows(row):
    plot = row['地块']
    area = row['种植面积']
    land_area = land_area_dict.get(plot, 0)  # 获取地块的面积，默认为0以防止 KeyError
    # 忽略种植面积小于地块面积 15% 的行
    return area >= 0.1 * land_area

# 应用过滤条件
df_filtered_results = df_results[df_results.apply(filter_rows, axis=1)]

# 查看过滤后的结果
for index,row in df_filtered_results.iterrows():
    print(row)

年份           2024
地块             A1
季次              1
作物名称           黍子
种植面积    35.646547
Name: 0, dtype: object
年份           2024
地块             A1
季次              1
作物名称           大麦
种植面积    18.546671
Name: 1, dtype: object
年份           2024
地块             A1
季次              1
作物名称           荞麦
种植面积    25.806782
Name: 2, dtype: object
年份           2024
地块             A2
季次              1
作物名称           黍子
种植面积    10.646547
Name: 3, dtype: object
年份           2024
地块             A2
季次              1
作物名称           大麦
种植面积    18.546671
Name: 4, dtype: object
年份           2024
地块             A2
季次              1
作物名称           荞麦
种植面积    25.806782
Name: 5, dtype: object
年份          2024
地块            A3
季次             1
作物名称          大麦
种植面积    9.193218
Name: 6, dtype: object
年份           2024
地块             A3
季次              1
作物名称           荞麦
种植面积    25.806782
Name: 7, dtype: object
年份           2024
地块             A4
季次              1
作物名称           黍子
种植面积    27.646547
Name: 8, dt

In [12]:
from openpyxl import load_workbook

file_path = 'result2.xlsx'

book = load_workbook(file_path)

crop_names = df_filtered_results['作物名称'].unique().tolist()

plot_to_row = {
    'A1': 2, 'A2': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 7,
    'B1': 8, 'B2': 9, 'B3': 10, 'B4': 11, 'B5': 12, 'B6': 13,
    'B7': 14, 'B8': 15, 'B9': 16, 'B10': 17, 'B11': 18, 'B12': 19,
    'B13': 20, 'B14': 21,
    'C1': 22, 'C2': 23, 'C3': 24, 'C4': 25, 'C5': 26, 'C6': 27,
    'D1': 28, 'D2': 29, 'D3': 30, 'D4': 31, 'D5': 32, 'D6': 33,
    'D7': 34, 'D8': 35,
    'E1': 36, 'E2': 37, 'E3': 38, 'E4': 39, 'E5': 40, 'E6': 41,
    'E7': 42, 'E8': 43, 'E9': 44, 'E10': 45, 'E11': 46, 'E12': 47,
    'E13': 48, 'E14': 49, 'E15': 50, 'E16': 51,
    'F1': 52, 'F2': 53, 'F3': 54, 'F4': 55
}

# 地块编号 -> Excel 行号的映射表 (季次2，从56行开始)
plot_to_row_season2 = {
    'D1': 56, 'D2': 57, 'D3': 58, 'D4': 59, 'D5': 60, 'D6': 61, 'D7': 62, 'D8': 63,
    'E1': 64, 'E2': 65, 'E3': 66, 'E4': 67, 'E5': 68, 'E6': 69, 'E7': 70, 'E8': 71,
    'E9': 72, 'E10': 73, 'E11': 74, 'E12': 75, 'E13': 76, 'E14': 77, 'E15': 78, 'E16': 79,
    'F1': 80, 'F2': 81, 'F3': 82, 'F4': 83
}

# 作物名称 -> Excel 列号的映射表
crop_to_col = {
    '黄豆': 3, '黑豆': 4, '红豆': 5, '绿豆': 6, '爬豆': 7, '小麦': 8, '玉米': 9, '谷子': 10, '高粱': 11,
    '黍子': 12, '荞麦': 13, '南瓜': 14, '红薯': 15, '莜麦': 16, '大麦': 17, '水稻': 18, '豇豆': 19, '刀豆': 20,
    '芸豆': 21, '土豆': 22, '西红柿': 23, '茄子': 24, '菠菜': 25, '青椒': 26, '菜花': 27, '包菜': 28, '油麦菜': 29,
    '小青菜': 30, '黄瓜': 31, '生菜': 32, '辣椒': 33, '空心菜': 34, '黄心菜': 35, '芹菜': 36, '大白菜': 37, '白萝卜': 38,
    '红萝卜': 39, '榆黄菇': 40, '香菇': 41, '白灵菇': 42, '羊肚菌': 43
}


def get_y_coord(plot, season):
    if season == 1:
        return plot_to_row.get(plot,1)  # 获取季次1对应的行号
    elif season == 2:
        return plot_to_row_season2.get(plot, 56)  # 获取季次2对应的行号，默认从第56行开始
# 填充表格为0的函数
def fill_sheet_with_zeros(sheet):
    for row in range(2, 84):  # 遍历行（C2 到 AP83）
        for col in range(3, 44):  # 遍历列（C 到 AP）
            sheet.cell(row=row, column=col).value = 0  # 填入0
# 遍历所有年份的表格，先填充为0
for year in range(2024, 2031):  # 假设年份从2024到2030
    sheet_name = str(year)
    sheet = book[sheet_name]
    fill_sheet_with_zeros(sheet)  # 将C2到AP83单元格填充为0

for _, row in df_filtered_results.iterrows():
    year = row['年份']
    plot = row['地块']
    season = row['季次']
    crop = row['作物名称']
    area = row['种植面积']
    sheet_name = str(year)
    sheet = book[sheet_name]
    # 根据地块和季次确定行数（Y 坐标）
    if season == 2 and plot[0] in ['A','B','C']:
        continue
    y_coord = get_y_coord(plot, season)
        # 根据作物名称确定列数（X 坐标）
    x_coord = crop_to_col.get(crop)  # 通过作物名称获取列号
        # 将种植面积写入对应的单元格
    sheet.cell(row=y_coord, column=x_coord).value = area
        # 保存修改后的 Excel 文件
book.save(file_path)
book.close()

print("结果已成功写入 Excel 文件！")

结果已成功写入 Excel 文件！


In [14]:
# 将结果保存为 Excel 文件
df_results.to_excel('crop_planting_results_2.xlsx', index=False)

print("Results saved to crop_planting_results_2.xlsx")

Results saved to crop_planting_results_2.xlsx
